In [1]:
# Import Libraries
import pandas as pd
import hvplot.pandas
import holoviews as hv 
import folium


# Define urls for data
bangor_temp_url = (
    'https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/city/time-'
    'series/USW00014606/tavg/1/7/1895-2023.csv?base_prd=true&begbaseyear=1991&endbaseyear=2020')

caribou_temp_url = (
    'https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/city/time-series'
    '/USW00014607/tavg/1/8/1953-2022.csv?base_prd=true&begbaseyear=1901&endbaseyear=2000')

portland_temp_url = (
    'https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/city/time-series'
    '/USW00014764/tavg/1/8/1953-2022.csv?base_prd=true&begbaseyear=1901&endbaseyear=2000')

maine_temp_url = ('https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/'
                  'statewide/time-series/17/tavg/1/8/1953-2022.csv?base_prd=true&'
                  'begbaseyear=1901&endbaseyear=2000')

# Read in Data
bangor_temp_df = pd.read_csv(bangor_temp_url, header = 4, names = ['year', 'temp_F', 'Anomaly']
                             )
caribou_temp_df = pd.read_csv(caribou_temp_url, header = 4, names = ['year', 'temp_F', 'Anomaly']
                             )
portland_temp_df = pd.read_csv(portland_temp_url, header = 4, names = ['year', 'temp_F', 'Anomaly']
                             )
maine_temp_df = pd.read_csv(maine_temp_url, header = 4, names = ['year', 'temp_F', 'Anomaly']
                             )

# Convert Date to Include Year Only
bangor_temp_df.year  = pd.to_datetime(bangor_temp_df.year, format = '%Y%m').dt.year
caribou_temp_df.year  = pd.to_datetime(caribou_temp_df.year, format = '%Y%m').dt.year
portland_temp_df.year  = pd.to_datetime(portland_temp_df.year, format = '%Y%m').dt.year
maine_temp_df.year  = pd.to_datetime(maine_temp_df.year, format = '%Y%m').dt.year

# # Merge city data
# maine_city_df = pd.merge(bangor_temp_df, caribou_temp_df, on="year")
# maine_city_df = pd.merge(maine_city_df, portland_temp_df, on="year")
# maine_city_df = maine_city_df.drop(columns=['Anomaly_x', 'Anomaly_y', 'Anomaly'])

# Calculate Temperature in Celsius
# bangor_temp_df['temp_C'] = (bangor_temp_df['temp_F'] - 32) * 5 / 9


# Site Information
This page displays annual average temperature data for Maine (statewide), as well as city-level data for the three cities in Maine with sites that report temperature data to NOAA National Centers for Environmental Information: Portland, Bangor, and Caribou. Data is from 1953 to 2022 as 1953 is the earliest date with data available for all three cities


In [2]:
# Map code adapted from: https://towardsdatascience.com/creating-a-simple-map-with-folium-and-python-4c083abfff94
# Define Latitute and Longitude
# List of ME Locations
maine_loc_list = [['Bangor', 44.8016, -68.7712], ['Portland', 43.6591, -70.2568], ['Caribou', 46.8640, -67.9980]]
  
# Create the pandas DataFrame of ME Locations from lists
maine_loc_df = pd.DataFrame(maine_loc_list, columns=['Site', 'Lat', 'Lon'])

# Create Map of ME Sites
site_ME_map = folium.Map(location=[maine_loc_df.Lat.mean(), 
                           maine_loc_df.Lon.mean()], 
                           zoom_start=7, control_scale=True)

# Add Markers of ME Sites
for index, maine_loc_df in maine_loc_df.iterrows():
    folium.Marker([maine_loc_df["Lat"], maine_loc_df["Lon"]], popup=maine_loc_df["Site"]).add_to(site_ME_map)

site_ME_map


# Data Citation
NOAA National Centers for Environmental information, Climate at a Glance: City Time Series, published September 2023, retrieved on September 8, 2023 from https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/city/time-series

# Average Annual Temperatures (°F) in Maine (1953 to 2022)

### Average annual temperatures in Maine have increased over time.

In [3]:
# Slope for ME State Data
slope = hv.Slope.from_scatter(hv.Scatter(maine_temp_df))

# Plot for ME State Data
maine_plt = maine_temp_df.hvplot(
    x='year', y='temp_F',
    title="Average Annual Temperature (°F) in Maine (1953 - 2022)",
    xlabel='Year', ylabel='Temperature (°F)', subplots=True, width=1050, color="#204060") * slope.opts(color="#b30000")

maine_plt

/opt/conda/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/opt/conda/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Overlay
   .Curve.I      :Curve   [year]   (temp_F)
   .Annotation.I :Slope   [x,y]

### Average annual temperatures in all three of Maine's cities with NECI sites (Bangor, Caribou, Portland) have increased over time.

In [4]:
# Plot Average Temperature across 3 Maine Cities
bangor_slope = hv.Slope.from_scatter(hv.Scatter(bangor_temp_df))
caribou_slope = hv.Slope.from_scatter(hv.Scatter(caribou_temp_df))
portland_slope = hv.Slope.from_scatter(hv.Scatter(portland_temp_df))

# maine_city_df.hvplot(
#      x='year', y=['Bangor, ME', 'Caribou, ME', 'Portland, ME'], 
#      xlabel='Year', ylabel='Temperature (°F)', subplots=True, width=350) 

# Define plots for each city
bangor_plt = bangor_temp_df.hvplot(
     x='year', y='temp_F', 
     title="Bangor, ME",
     xlabel='Year', 
     ylabel='Temperature (°F)', 
     subplots=True, 
     ylim=[58, 73],
     width=350) * bangor_slope.opts(color="#ff7733")

caribou_plt = caribou_temp_df.hvplot(
     x='year', y='temp_F',
     title="Caribou, ME",
     ylim=[58, 73],
     xlabel='Year', 
     ylabel='Temperature (°F)', 
     subplots=True, 
     width=350) * caribou_slope.opts(color="#ff7733")

portland_plt = portland_temp_df.hvplot(
     x='year', y='temp_F',
     title="Portland, ME",
     ylim=[58, 73],
     xlabel='Year', 
     ylabel='Temperature (°F)', 
     subplots=True, 
     width=350) * portland_slope.opts(color="#ff7733")

# Display 3 City Plots
combined_plots = portland_plt + bangor_plt + caribou_plt

combined_plots

/opt/conda/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/opt/conda/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/opt/conda/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  re

:Layout
   .Overlay.I   :Overlay
      .Curve.I      :Curve   [year]   (temp_F)
      .Annotation.I :Slope   [x,y]
   .Overlay.II  :Overlay
      .Curve.I      :Curve   [year]   (temp_F)
      .Annotation.I :Slope   [x,y]
   .Overlay.III :Overlay
      .Curve.I      :Curve   [year]   (temp_F)
      .Annotation.I :Slope   [x,y]

In [5]:
%%capture
%%bash
jupyter nbconvert temperatureAnalysis.ipynb --to html --no-input